In [1]:
from compas.datastructures import Mesh
from compas.datastructures import subdivision as sd
import ipyvolume as ipv
from utilities import draw_compas_mesh
from compas.geometry import Vector

In [2]:
mesh = Mesh.from_polyhedron(12)
mesh.summary()


Mesh summary

- name: Mesh
- vertices: 20
- edges: 30
- faces: 12
- vertex degree: 3/3
- face degree: 5/5




In [3]:
mesh2 = sd.mesh_subdivide_tri(mesh)
mesh3 = sd.trimesh_subdivide_loop(mesh2)

In [4]:
draw_compas_mesh(mesh3)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [5]:
subd = mesh3.copy()
height = .2

for fkey in mesh3.faces():
    centroid = mesh3.face_centroid(fkey)
    centroid_vector = Vector(*centroid)
    normal = mesh3.face_normal(fkey)
    normal_vector = Vector(*normal)
    new_vertex = centroid_vector + normal_vector * height
    subd.insert_vertex(fkey, xyz=new_vertex)

draw_compas_mesh(subd)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [6]:
def mesh_subdivide_pyramid(mesh, k=1, height=1.0):
    """Subdivide a mesh using insertion of vertex at centroid + height * face normal.

    Parameters
    ----------
    mesh : Mesh
        The mesh object that will be subdivided.
    k : int, optional
        The number of levels of subdivision. Default is ``1``.
    height : float, optional
        The distance of the new vertex to the face.

    Returns
    -------
    Mesh
        A new subdivided mesh.
    """
    if k != 1:
        raise NotImplementedError
    
    subd = mesh.copy()
 
    for fkey in mesh.faces():
        centroid = mesh.face_centroid(fkey)
        centroid_vector = Vector(*centroid)
        normal = mesh.face_normal(fkey)
        normal_vector = Vector(*normal)
        new_vertex = centroid_vector + normal_vector * height
        subd.insert_vertex(fkey, xyz=new_vertex)
        
    return subd




In [7]:
our_mesh = mesh_subdivide_pyramid(mesh3, height=0.3)
draw_compas_mesh(our_mesh)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [8]:
def mesh_subdivide_tapered(mesh, k=1, height=1.0, ratio=0.5):
    """
    """
    if ratio == int(1.):
        return mesh_subdivide_pyramid(mesh, k, height)
    
    
    #ubd = mesh.copy()
    
    for _ in range(k):
    
        subd = mesh.copy()
        for fkey in mesh.faces():
            centroid = mesh.face_centroid(fkey)
            centroid_vector = Vector(*centroid)
            normal = mesh.face_normal(fkey)
            normal_vector = Vector(*normal)
            normal_vector *= height

            face_verts = mesh.face_vertices(fkey)
            new_verts = []

            for v in face_verts:
                v_coords = mesh.vertex_coordinates(v)
                v_vector = Vector(*v_coords)

                vert_to_center = centroid_vector - v_vector
                vert_to_center *= ratio
                new_vertex = v_vector + vert_to_center + normal_vector
                x, y, z = new_vertex
                new_verts.append(subd.add_vertex(x=x, y=y, z=z))

            for i, v in enumerate(face_verts):
                next_v = face_verts[(i+1) % len(face_verts)]
                new_v = new_verts[i]
                next_new_v = new_verts[(i+1) % len(face_verts)]
                new_face_key = subd.add_face([v, next_v, next_new_v, new_v])
                subd.set_face_attribute(new_face_key, 'material', 'frame')

            top_face_key = subd.add_face(new_verts)
            subd.set_face_attribute(top_face_key, 'material', 'glass')
            del subd.face[fkey]
        mesh = subd
        
    return subd
            
 #

In [9]:
mesh4 = sd.mesh_subdivide_catmullclark(mesh3)
#tapered_mesh = mesh_subdivide_tapered(mesh2, k=5, height=0.3, ratio=.4)
#draw_compas_mesh(tapered_mesh)


# tapered_mesh.get_faces_attribute(tapered_mesh.faces(), 'material')

In [24]:
face_verts_list = []

mesh.faces()

for f_vert in mesh3.faces():
    face_verts_list.append(mesh3.face_vertices(f_vert))


[[33, 0, 32], [32, 13, 96], [96, 20, 33], [32, 96, 33], [96, 13, 85], [85, 11, 86], [86, 20, 96], [85, 86, 96], [86, 11, 38], [38, 1, 39], [39, 20, 86], [38, 39, 86], [39, 1, 40], [40, 3, 50], [50, 20, 39], [40, 50, 39], [50, 3, 34], [34, 0, 33], [33, 20, 50], [34, 33, 50], [35, 0, 34], [34, 3, 51], [51, 21, 35], [34, 51, 35], [51, 3, 44], [44, 2, 45], [45, 21, 51], [44, 45, 51], [45, 2, 46], [46, 8, 73], [73, 21, 45], [46, 73, 45], [73, 8, 74], [74, 10, 81], [81, 21, 73], [74, 81, 73], [81, 10, 36], [36, 0, 35], [35, 21, 81], [36, 35, 81], [37, 0, 36], [36, 10, 82], [82, 22, 37], [36, 82, 37], [82, 10, 83], [83, 18, 116], [116, 22, 82], [83, 116, 82], [116, 18, 90], [90, 12, 91], [91, 22, 116], [90, 91, 116], [91, 12, 92], [92, 13, 97], [97, 22, 91], [92, 97, 91], [97, 13, 32], [32, 0, 37], [37, 22, 97], [32, 37, 97], [42, 1, 41], [41, 4, 53], [53, 23, 42], [41, 53, 42], [53, 4, 54], [54, 7, 69], [69, 23, 53], [54, 69, 53], [69, 7, 47], [47, 2, 48], [48, 23, 69], [47, 48, 69], [48, 2,

Signature: sd.mesh_subdivide_doosabin(mesh, k=1, fixed=None)
Docstring:
Subdivide a mesh following the doo-sabin scheme.

Parameters
----------
mesh : Mesh
    The mesh object that will be subdivided.
k : int
    Optional. The number of levels of subdivision. Default is ``1``.
fixed : list
    Optional. A list of fixed vertices. Default is ``None``.

Returns
-------
Mesh
    A new subdivided mesh.
File:      c:\tools\miniconda3\envs\mas1920t1\lib\site-packages\compas\datastructures\mesh\subdivision.py
Type:      function


In [42]:

first_vertices = [v[0] for v in face_verts_list[0]]

dosa = sd.mesh_subdivide_doosabin(mesh3, k=5, fixed=mesh3.vertex_coordinates(first_vertices))
draw_compas_mesh(dosa)


TypeError: 'int' object is not subscriptable

In [10]:
from utilities import export_obj_by_attribute
#export_obj_by_attribute('spacestation.obj', tapered_mesh, 'material')